In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
findspark.init()
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [4]:
file = "/content/sample_data/california_housing_test.csv"
spark.read.format("csv").load(file)

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string]

In [8]:
!pip install lorem


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
import lorem
s = lorem.paragraph()
s = s.split()

In [30]:
rdd = spark.sparkContext.parallelize(s)
rdd

ParallelCollectionRDD[43] at readRDDFromFile at PythonRDD.scala:274

In [31]:
rdd.count()

24

In [33]:
rdd.distinct().collect()

['Ipsum',
 'quaerat',
 'porro.',
 'Quiquia',
 'magnam',
 'quiquia',
 'quisquam.',
 'amet',
 'modi',
 'porro',
 'sit',
 'non',
 'Dolore',
 'dolor',
 'adipisci',
 'aliquam',
 'velit.',
 'Porro',
 'dolore.',
 'Tempora',
 'voluptatem.']

In [36]:
rdd.collect()

['Ipsum',
 'sit',
 'quaerat',
 'porro.',
 'Quiquia',
 'non',
 'magnam',
 'quiquia',
 'quisquam.',
 'Dolore',
 'amet',
 'dolor',
 'adipisci',
 'modi',
 'aliquam',
 'velit.',
 'Porro',
 'porro',
 'dolor',
 'dolore.',
 'Tempora',
 'magnam',
 'sit',
 'voluptatem.']

In [38]:
rdd.filter(lambda word: word.lower().startswith('d')).collect()

['Dolore', 'dolor', 'dolor', 'dolore.']

In [39]:
rdd.map(lambda x: x.upper()).collect()

['IPSUM',
 'SIT',
 'QUAERAT',
 'PORRO.',
 'QUIQUIA',
 'NON',
 'MAGNAM',
 'QUIQUIA',
 'QUISQUAM.',
 'DOLORE',
 'AMET',
 'DOLOR',
 'ADIPISCI',
 'MODI',
 'ALIQUAM',
 'VELIT.',
 'PORRO',
 'PORRO',
 'DOLOR',
 'DOLORE.',
 'TEMPORA',
 'MAGNAM',
 'SIT',
 'VOLUPTATEM.']

In [40]:
rdd.reduce(lambda y ,z: y if len(y)>len(z) else z)

'voluptatem.'

In [51]:
rdd.map(lambda x: (x.title(), x)).sortByKey(False).collect()

[('Voluptatem.', 'voluptatem.'),
 ('Velit.', 'velit.'),
 ('Tempora', 'Tempora'),
 ('Sit', 'sit'),
 ('Sit', 'sit'),
 ('Quisquam.', 'quisquam.'),
 ('Quiquia', 'Quiquia'),
 ('Quiquia', 'quiquia'),
 ('Quaerat', 'quaerat'),
 ('Porro.', 'porro.'),
 ('Porro', 'Porro'),
 ('Porro', 'porro'),
 ('Non', 'non'),
 ('Modi', 'modi'),
 ('Magnam', 'magnam'),
 ('Magnam', 'magnam'),
 ('Ipsum', 'Ipsum'),
 ('Dolore.', 'dolore.'),
 ('Dolore', 'Dolore'),
 ('Dolor', 'dolor'),
 ('Dolor', 'dolor'),
 ('Amet', 'amet'),
 ('Aliquam', 'aliquam'),
 ('Adipisci', 'adipisci')]

In [52]:
import requests

1. Get the temperatures for the next week

In [ ]:
country = "United States"
zipcode = "21220"
locreq = f"https://nominatim.openstreetmap.org/search?postalcode=<{zipcode}>&country=<{country}>&format=json&polygon_geojson=1&addressdetails=1"
loc = requests.get(locreq).json()[0]
loc

In [ ]:
latitude = loc["lat"]
longitude = loc["lon"]
url = f"https://api.weather.gov/points/{latitude},{longitude}"
req = requests.get(url).json()['properties']['forecastHourly']
data = requests.get(req).json()
data['properties']['periods']

2. Create an RDD of the temperatures

In [78]:
temps = spark.sparkContext.parallelize(data['properties']['periods'])
tempF = temps.map(lambda x: (x["startTime"],x["temperature"]))
tempF.first()

('2022-09-22T13:00:00-04:00', 72)

3. Create an RDD of the temps in Celsius

In [77]:
tempC = temps.map(lambda x: (x['startTime'], ((x["temperature"]-32)*(5/9))))
tempC.first()

('2022-09-22T13:00:00-04:00', 22.22222222222222)

4. Find all temps that are greater then 13 C

In [ ]:
tempC_13 = tempC.filter(lambda x: x[1] > 13).collect()
tempC_13

In [87]:
temp_time = temps.map(lambda x: (x["temperature"],x["startTime"]))

5. Map the RDD to a tuple of days, temp

In [ ]:
temp_time.collect()

6. Sort the RDD by temp

In [93]:
temp_time.sortByKey().collect()

[(49, '2022-09-24T04:00:00-04:00'),
 (49, '2022-09-24T05:00:00-04:00'),
 (49, '2022-09-24T06:00:00-04:00'),
 (49, '2022-09-24T07:00:00-04:00'),
 (50, '2022-09-23T06:00:00-04:00'),
 (50, '2022-09-23T07:00:00-04:00'),
 (50, '2022-09-24T03:00:00-04:00'),
 (51, '2022-09-23T05:00:00-04:00'),
 (51, '2022-09-24T02:00:00-04:00'),
 (51, '2022-09-24T08:00:00-04:00'),
 (52, '2022-09-23T04:00:00-04:00'),
 (52, '2022-09-23T08:00:00-04:00'),
 (52, '2022-09-24T01:00:00-04:00'),
 (53, '2022-09-23T03:00:00-04:00'),
 (53, '2022-09-24T00:00:00-04:00'),
 (54, '2022-09-23T02:00:00-04:00'),
 (54, '2022-09-23T09:00:00-04:00'),
 (54, '2022-09-23T23:00:00-04:00'),
 (54, '2022-09-28T04:00:00-04:00'),
 (54, '2022-09-28T05:00:00-04:00'),
 (54, '2022-09-28T06:00:00-04:00'),
 (54, '2022-09-28T07:00:00-04:00'),
 (55, '2022-09-23T01:00:00-04:00'),
 (55, '2022-09-23T22:00:00-04:00'),
 (55, '2022-09-24T09:00:00-04:00'),
 (55, '2022-09-28T03:00:00-04:00'),
 (56, '2022-09-23T00:00:00-04:00'),
 (56, '2022-09-23T10:00:00-0

In [96]:
import pyspark.pandas as ps
import seaborn as sns
taxis = sns.load_dataset("taxis")

1. Find the total and average per payment type
2. Find the total number of rides that were picked up and dropped off in the same borough
3. Find the average amount of passangers per pickup borough
4. Find the greatest distence per pickup borough, and the greatest distence with the same pickup and dropoff borough
5. Find the longest and shortest rides that included a toll, and did not include a toll

In [98]:
df = ps.DataFrame(taxis)
df.head()

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan
3,2019-03-10 01:23:59,2019-03-10 01:49:51,1,7.70,27.0,6.15,0.0,36.95,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan
4,2019-03-30 13:27:42,2019-03-30 13:37:14,3,2.16,9.0,1.10,0.0,13.40,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan


Find the total and average per payment type

In [99]:
df.groupby("payment").mean()

,passengers,distance,fare,tip,tolls,total
payment,,,,,,
cash,1.552428,2.540944,11.592991,0.000000,0.187445,14.676849
credit card,1.538781,3.221842,13.694750,2.781805,0.380448,20.071248


In [100]:
df.groupby("payment").sum()

,passengers,distance,fare,tip,tolls,total
payment,,,,,,
cash,2813,4604.19,21006.50,0.00,339.65,26594.45
credit card,7043,14746.37,62680.87,12732.32,1741.31,91866.10


Find the total number of rides that were picked up and dropped off in the same borough

In [102]:
df[df["pickup_borough"] == df["dropoff_borough"]]["pickup"].count()

5582

Find the average amount of passangers per pickup borough

In [109]:
df.groupby("pickup_borough").mean()[["passengers"]]

,passengers
pickup_borough,
Queens,1.523592
Brooklyn,1.310705
Manhattan,1.566059
Bronx,1.191919


Find the greatest distence per pickup borough, and the greatest distence with the same pickup and dropoff borough

In [111]:
df.groupby("pickup_borough").max()[["distance"]]

,distance
pickup_borough,
Queens,36.70
Brooklyn,25.51
Manhattan,28.30
Bronx,23.61


In [112]:
df[df["pickup_borough"] == df["dropoff_borough"]].groupby("pickup_borough").max()[["distance"]]

,distance
pickup_borough,
Queens,36.70
Brooklyn,14.09
Manhattan,13.27
Bronx,23.61


Find the longest and shortest rides that included a toll, and did not include a toll

In [115]:
#longest ride including tolls
df[df["distance"] == df[df["tolls"] > 0].max()["distance"]]

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
5364,2019-03-17 16:59:17,2019-03-17 18:04:08,2,36.7,150.0,0.0,24.02,174.82,yellow,cash,JFK Airport,JFK Airport,Queens,Queens


In [123]:
#shortest ride including tolls
df[(df["distance"] == df[df["tolls"] > 0].min()["distance"]) & (df["tolls"] > 0)]

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
622,2019-03-12 19:52:36,2019-03-12 19:52:55,3,0.0,120.0,33.20,12.50,166.00,yellow,credit card,None,None,None,None
3095,2019-03-19 22:20:25,2019-03-19 22:20:38,1,0.0,52.0,0.00,5.76,61.06,yellow,cash,Midtown Center,Midtown Center,Manhattan,Manhattan
3221,2019-03-23 17:11:33,2019-03-23 17:11:50,2,0.0,52.0,12.21,5.76,73.27,yellow,credit card,Clinton East,Clinton East,Manhattan,Manhattan
5433,2019-03-01 17:53:56,2019-03-01 17:54:19,1,0.0,52.0,13.66,8.50,81.96,yellow,credit card,TriBeCa/Civic Center,TriBeCa/Civic Center,Manhattan,Manhattan


In [124]:
# longest ride without tolls
df[(df["distance"] == df[df["tolls"] == 0].max()["distance"]) & (df["tolls"] == 0)]

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
4050,2019-03-13 14:05:19,2019-03-13 15:01:16,6,33.76,143.5,0.0,0.0,144.3,yellow,cash,LaGuardia Airport,None,Queens,None


In [125]:
#shortest ride without tolls
df[(df["distance"] == df[df["tolls"] == 0].min()["distance"]) & (df["tolls"] == 0)]

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
42,2019-03-30 23:59:14,2019-03-30 23:59:17,1,0.0,80.0,20.08,0.0,100.38,yellow,credit card,None,None,None,None
120,2019-03-21 17:21:44,2019-03-21 17:21:49,1,0.0,2.5,0.00,0.0,4.30,yellow,cash,JFK Airport,JFK Airport,Queens,Queens
670,2019-03-08 04:50:48,2019-03-08 04:51:07,1,0.0,2.5,0.00,0.0,6.30,yellow,cash,Lenox Hill East,Lenox Hill East,Manhattan,Manhattan
970,2019-03-29 00:02:00,2019-03-29 00:02:05,2,0.0,45.0,5.00,0.0,50.30,yellow,credit card,None,None,None,None
1080,2019-03-04 14:17:05,2019-03-04 14:17:13,1,0.0,2.5,0.00,0.0,3.30,yellow,cash,JFK Airport,JFK Airport,Queens,Queens
1207,2019-03-08 15:41:20,2019-03-08 15:41:23,1,0.0,2.5,0.00,0.0,5.80,yellow,None,Hudson Sq,Hudson Sq,Manhattan,Manhattan
1482,2019-03-10 20:27:25,2019-03-10 20:28:33,1,0.0,52.0,16.59,0.0,71.89,yellow,credit card,Lincoln Square East,Lincoln Square East,Manhattan,Manhattan
1690,2019-03-22 06:24:14,2019-03-22 06:24:14,1,0.0,72.0,0.00,0.0,72.00,yellow,None,East New York,None,Brooklyn,None
1910,2019-03-25 14:37:56,2019-03-25 14:38:22,1,0.0,2.5,0.01,0.0,3.31,yellow,credit card,Queensbridge/Ravenswood,Queensbridge/Ravenswood,Queens,Queens
1929,2019-03-13 22:35:35,2019-03-13 22:35:49,1,0.0,2.5,0.00,0.0,3.80,yellow,None,JFK Airport,JFK Airport,Queens,Queens
